## Imports / Path

In [1]:
import exifread
import pandas as pd
from exif import Image
from os import walk
path = './'

## Functions 

In [6]:
def get_exif_data(image_file):
    with open(image_file, 'rb') as f:
        exif_tags = exifread.process_file(f, details=False)
        gps_tags = dict()
        for tag in exif_tags.keys():
            if tag in ('GPS GPSLatitude', 'GPS GPSLatitudeRef', 'GPS GPSLongitude', 'GPS GPSLongitudeRef'):
                gps_tags[tag] = exif_tags[tag]
    return gps_tags

def _get_if_exist(data, key):
    if key in data:
        return data[key]

    return None

def _convert_to_degress(value):
    """
    Helper function to convert the GPS coordinates stored in the EXIF to degress in float format
    :param value:
    :type value: exifread.utils.Ratio
    :rtype: float
    """
    d = float(value.values[0].num) / float(value.values[0].den)
    m = float(value.values[1].num) / float(value.values[1].den)
    s = float(value.values[2].num) / float(value.values[2].den)

    return d + (m / 60.0) + (s / 3600.0)

def get_exif_location(exif_data):
    """
    Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)
    """
    lat = None
    lon = None

    gps_latitude = _get_if_exist(exif_data, 'GPS GPSLatitude')
    gps_latitude_ref = _get_if_exist(exif_data, 'GPS GPSLatitudeRef')
    gps_longitude = _get_if_exist(exif_data, 'GPS GPSLongitude')
    gps_longitude_ref = _get_if_exist(exif_data, 'GPS GPSLongitudeRef')

    if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
        lat = _convert_to_degress(gps_latitude)
        if gps_latitude_ref.values[0] != 'N':
            lat = 0 - lat

        lon = _convert_to_degress(gps_longitude)
        if gps_longitude_ref.values[0] != 'E':
            lon = 0 - lon

    return lat, lon

def get_exif_date(file):
    """
    Returns the datetime from exif data 
    """
    with open(file, 'rb') as src:
        img = Image(src)
    date = img.datetime_original
    return date

def wall_data(filenames, path, year):
    """
    Create a dataframe w/ lat, long, date, and filename 
    """
    holder = pd.DataFrame(columns=['Filename', 'Lat', 'Long', 'Date'])
    counter = 0
    for file in filenames:
        if file != '.DS_Store':
            lat, long = get_exif_location(get_exif_data(path+file))
            date = get_exif_date(path+file)
            holder.at[counter,:] = file, lat, long, date
            counter += 1
    holder['Filename'] = f'media/{year}/' + holder['Filename'].astype(str)
    return holder

## Run
Load filenames, process everything vai the wall_data function, then export the csv 


In [7]:
filenames = next(walk(path+'PHOTOS/2020'), (None, None, []))[2]  # [] if no file
export = wall_data(filenames, './PHOTOS/2020/', '2020')
export.to_csv('2020.csv')

In [8]:
export

,Filename,Lat,Long,Date
0,media/2020/IMG_4133.jpeg,44.493031,11.347819,2020:12:25 10:36:35
1,media/2020/IMG_4164.jpeg,44.493694,11.358617,2020:12:31 10:42:06
2,media/2020/IMG_4021.jpeg,44.494236,11.351339,2020:12:07 10:47:36
3,media/2020/IMG_4037.jpeg,44.497608,11.348506,2020:12:08 10:29:00
4,media/2020/IMG_4125.jpeg,44.495139,11.351567,2020:12:23 09:33:28
...,...,...,...,...
94,media/2020/IMG_4002.jpeg,44.497692,11.354297,2020:11:30 13:24:05
95,media/2020/IMG_4022.jpeg,44.494158,11.351625,2020:12:07 10:48:02
96,media/2020/IMG_4167.jpeg,44.500094,11.361867,2020:12:31 11:35:24
97,media/2020/IMG_4063.jpeg,44.494867,11.351869,2020:12:10 13:55:29
